In [1]:
import os
import plotly.express as px
import plotly.io as pio
import pandas as pd

if os.path.basename(os.getcwd()) != 'in-the-wild-verification':
    os.chdir(os.path.dirname(os.getcwd()))

print(os.getcwd())

/Users/Bianka/Desktop/szkoła/biometria/in-the-wild-verification


In [12]:
def plot_line(x, y, x_label, y_label):
    fig = px.line(x = x, y = y,
              labels={'x':x_label,
                'y': y_label,
                },)
    fig.update_layout(height=400, width=800, showlegend=False, font=dict(family="CMU Serif",size=14))
    config = {
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'height': 600, 'width': 900, 'scale':6
    }}
    
    fig.show(config=config)
    pio.write_image(fig, f"loss.png", scale=6, width=900, height=500)

In [9]:
results = pd.read_csv("speech_id_checkpoint/Stats", sep='\t', index_col=False)

results.head()

/var/folders/y5/sg4lr5xn7fv5ry3py7wvzj240000gn/T/ipykernel_44682/181831623.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  results = pd.read_csv("speech_id_checkpoint/Stats", sep='\t', index_col=False)


,Date,Epoch,Loss,Total loss
0,Sun Jun 2 00:19:37 2024,Epoch:1[28/191],Iteration:28,27.1325
1,Sun Jun 2 00:24:15 2024,Epoch:1[56/191],Iteration:56,25.7418
2,Sun Jun 2 00:28:45 2024,Epoch:1[84/191],Iteration:84,36.0434
3,Sun Jun 2 00:32:59 2024,Epoch:1[112/191],Iteration:112,24.8208
4,Sun Jun 2 00:38:14 2024,Epoch:1[140/191],Iteration:140,12.2712


In [13]:
tloss = results['Total loss'].tolist()
tloss_epoch = [sum(tloss[i:i+6])/6 for i in range(0,len(tloss),6)][:-1]
epochs = range(0, int((len(results)-1)/6))

plot_line(epochs, tloss_epoch, 'Epoka', 'Całkowity koszt')

In [3]:
from speech_embedder_net import SpeechEmbedder
from speech_model.utils import mfccs_and_spec
import torch

class SpekerVerifier:
    """Class for loading the model and computing speech embeddings"""
    def __init__(
            self,
            model_path: str = 'speech_id_checkpoint/ckpt_epoch_180_batch_id_191.pth'
            ) -> None:
        self.model = SpeechEmbedder()
        self.model.load_state_dict(torch.load(model_path))
        self.model.eval()
        pass

    def compute_embedding(self, audio_input):
        _, mel_db, _ = mfccs_and_spec(audio_input, wav_process = True)
        mel_db = torch.Tensor(mel_db)
        enrollment_batch = torch.reshape(mel_db, (1, mel_db.size(0), mel_db.size(1)))
        embedding = self.model(torch.Tensor(enrollment_batch))
        return embedding

In [4]:
sample_file = 'split_data/train/id10'